In [3]:
# Importing Libraries
import time
import requests
import pandas as pd
from bs4 import BeautifulSoup
from functools import reduce

# Function to merge the datas
def merge_mult_dfs(dfs):
    return reduce(lambda left,right: pd.merge(left,right,on='Player'), dfs)

# Creating empty list to store data
all_players=[]
all_keeper=[]

# Loading the URL
standing_url="https://fbref.com/en/comps/1/World-Cup-Stats"
data=requests.get(standing_url)
League_Table=pd.read_html(data.text, match="League Table")[0]
soup=BeautifulSoup(data.text)
standing_table=soup.select("table.stats_table")[-1]
links=standing_table.find_all("a")
links=[l.get("href") for l in links]
links=[l for l in links if "/squads/" in l]
team_urls=[f"https://fbref.com{l}" for l in links]

for team in team_urls:
    team_name=team.split("/")[-1].replace("-Men-Stats","").replace("-"," ")
    data=requests.get(team)
    shoot=pd.read_html(data.text, match="Shooting")[0]
    shoot.columns=shoot.columns.droplevel()
    keep=pd.read_html(data.text, match="Goalkeeping")[0]
    keep.columns=keep.columns.droplevel()
    Pass=pd.read_html(data.text, match="Passing")[0]
    Pass.columns=Pass.columns.droplevel()
    PaT=pd.read_html(data.text, match="Pass Types")[0]
    PaT.columns=PaT.columns.droplevel()
    goal=pd.read_html(data.text, match="Goal and Shot Creation")[0]
    goal.columns=goal.columns.droplevel()
    defen=pd.read_html(data.text, match="Defensive Actions")[0]
    defen.columns=defen.columns.droplevel()
    pos=pd.read_html(data.text, match="Possession")[0]
    pos.columns=pos.columns.droplevel()
    stats=pd.read_html(data.text, match="Miscellaneous Stats")[0]
    stats.columns=stats.columns.droplevel()
    play=pd.read_html(data.text, match="Playing Time")[0]
    play.columns=play.columns.droplevel()
    stand=pd.read_html(data.text,match="Standard Stats")[0]
    stand.columns=stand.columns.droplevel()

    n_stand=['Player', 'Pos', 'Age', 'Club', 'MP', 'Starts', 'Min', '90s', 'Gls',
        'Ast', 'G-PK', 'PK','CrdY', 'CrdR']
    _stand=stand[n_stand]

    n_shoot=['Player','Sh', 'SoT', 'G/Sh', 'G/SoT']
    _shoot=shoot[n_shoot]

    n_keep=['Player', 'Pos', 'Age', 'MP', 'Starts', 'Min', '90s','SoTA','Saves', 'Save%', 'W', 'D', 'L', 'CS', 'CS%']
    _keep=keep[n_keep]

    n_ass=Pass.iloc[:,4:7]
    n_ass["Player"]=Pass['Player']

    n_defen=['Player','Tkl', 'TklW','Blocks', 'Int', 'Tkl+Int', 'Clr', 'Err']
    _defen=defen[n_defen]

    n_pos=["Player",'Touches','Succ', 'Att', 'Succ%']
    _pos=pos[n_pos]

    n_stats=['Player','Fls', 'Fld', 'Off', 'Crs', 'Int', 'TklW', 'PKwon', 'PKcon', 'OG','Won', 'Lost',]
    _stats=stats[n_stats]

    match=[_shoot,n_ass,_defen,_pos,_stats,_stand]
    team_match=merge_mult_dfs(match)

    team_match["Team"]=team_name
    new=team_match.query("Player != 'Squad Total' and Player !='Opponent Total'")

    Keeping=_keep.query("Player != 'Squad Total' and Player !='Opponent Total'")

    all_players.append(new)
    all_keeper.append(Keeping)

    time.sleep(3)

#Merge all data from each team
combined_players = pd.concat(all_players, ignore_index=True)
combined_keepers = pd.concat(all_keeper, ignore_index=True)

In [4]:
combined_keepers

,Player,Pos,Age,MP,Starts,Min,90s,SoTA,Saves,Save%,Save%,W,D,L,CS,CS%
0,Emiliano Martínez,GK,29.0,7,7,690,7.7,13,6,53.8,0.0,4,2,1,3,42.9
1,Hugo Lloris,GK,35.0,6,6,570,6.3,24,17,83.3,0.0,5,1,0,1,16.7
2,Steve Mandanda,GK,37.0,1,1,90,1.0,3,2,66.7,NaN,0,0,1,0,0.0
3,Dominik Livaković,GK,27.0,7,7,690,7.7,31,24,80.6,0.0,2,4,1,2,28.6
4,Yassine Bounou,GK,31.0,6,6,570,6.3,11,7,54.5,NaN,2,2,2,3,50.0
5,Munir,GK,33.0,1,1,90,1.0,3,3,100.0,NaN,1,0,0,1,100.0
6,Andries Noppert,GK,28.0,5,5,480,5.3,22,18,86.4,0.0,3,2,0,2,40.0
7,Jordan Pickford,GK,28.0,5,5,450,5.0,11,7,72.7,0.0,3,1,1,3,60.0
8,Alisson,GK,29.0,4,4,379,4.2,7,5,71.4,NaN,3,1,0,2,50.0
9,Ederson,GK,28.0,1,1,90,1.0,3,2,66.7,NaN,0,0,1,0,0.0


In [5]:
combined_players

,Player,Sh,SoT,G/Sh,G/SoT,Cmp,Att_x,Cmp%,Tkl,Tkl,...,Gls,Gls,Ast,Ast,G-PK,G-PK,PK,CrdY,CrdR,Team
0,Emiliano Martínez,0,0,NaN,NaN,120.0,187.0,64.2,0.0,0.0,...,0.0,0.00,0.0,0.00,0.0,0.00,0.0,1.0,0.0,Argentina
1,Lionel Messi,27,13,0.11,0.23,300.0,371.0,80.9,5.0,0.0,...,7.0,0.91,3.0,0.39,3.0,0.39,4.0,1.0,0.0,Argentina
2,Nicolás Otamendi,1,0,0.00,NaN,532.0,577.0,92.2,10.0,5.0,...,0.0,0.00,1.0,0.13,0.0,0.00,0.0,2.0,0.0,Argentina
3,Rodrigo De Paul,7,3,0.00,0.00,470.0,562.0,83.6,11.0,8.0,...,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.0,Argentina
4,Nahuel Molina,2,1,0.50,1.00,291.0,358.0,81.3,8.0,4.0,...,1.0,0.16,1.0,0.16,1.0,0.16,0.0,0.0,0.0,Argentina
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
675,Mohammed Waad,0,0,NaN,NaN,9.0,12.0,75.0,0.0,0.0,...,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.0,Qatar
676,Ali Assadalla,0,0,NaN,NaN,21.0,23.0,91.3,0.0,0.0,...,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.0,Qatar
677,Tarek Salman,0,0,NaN,NaN,7.0,10.0,70.0,1.0,0.0,...,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.0,Qatar
678,Ahmed Alaaeldin,0,0,NaN,NaN,0.0,2.0,0.0,0.0,0.0,...,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.0,Qatar


In [8]:
#Save the data to CSV files
combined_players.to_csv("combined_players.csv")
combined_keepers.to_csv("combined_keepers.csv")
